### CNN training

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support as prf

In [3]:
from PIL import Image, ExifTags, ImageOps, ImageDraw
from src import bbox2tlbr, sqrbbox, compute_IoU

In [4]:
import torch
from torchvision import transforms

### 1. Dataset & dataloader

- Dataset y dataloader &rarr; fer treballar la cpu i gpu en paral·let
- Avui veiem com entrenar la xarxa

In [5]:
from torch.utils.data import Dataset, DataLoader

#### Data

In [6]:
_imgRoot = '../mosquits/phase2_test/test/final/'

# El dataset l'hauríem de complimentar amb algunes imatges sense mosquit per tenir '??'
# _classes = ['aegypti', 'albopictus', 'anopheles', 'culex', 'culiseta', 'japonicus-koreicus', '??']
_classes = ['aegypti', 'albopictus', 'anopheles', 'culex', 'culiseta', 'japonicus-koreicus']

# El que comentava al final del anterior notebook
_imgSize = 512 # pretrained at 384

_imgNorm = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

# control image max-size 
Image.MAX_IMAGE_PIXELS = 201326592
Image.warnings.simplefilter('error', Image.DecompressionBombWarning)

In [7]:
class maDataset(Dataset):
    
    def __init__(self, csvDataFile):
        
        self.df = pd.read_csv(csvDataFile)
        self.transform = transforms.Compose([
                transforms.Resize((_imgSize, _imgSize)),
                transforms.ToTensor(),
                transforms.Normalize(_imgNorm[0], _imgNorm[1])
            ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
    
        # open image file
        row = self.df.iloc[idx]
        pilImg = Image.open('%s/%s' %(_imgRoot, row.img_fName))
        # crop image
        bbox = sqrbbox([(row.bbx_xtl, row.bbx_ytl), (row.bbx_xbr, row.bbx_ybr)], pilImg.size)
        pilImg = pilImg.crop(bbox)
        # transform to torch tensor image
        torchImg = self.transform(pilImg)

        return {'img_fName': row.img_fName, 'image' : torchImg, 'label': [_classes.index(row.class_label)]}
        # row class_label no són etiquetes alphanumeriques, ho convertim a numeric dins del index
        # Algo que ho converteixi a integers
        # si és la etiqueta 3, la neurona que s'ha d'activar més es la 3

- Quan volem fer una subclasse de la dataset torch, ha de tenir dos metodes: 
    - len 
    - getitems
    - definir els transforms
- Flexible size: 
    - les podem entrenar a 512 i fer inferencia a una altre
- Input size d'una determinada xarxa, en molts casos el que pasa es que aquesta mida és flexible.

In [8]:
# instantiate our custom dataset
csvDataFile = '../mosquits/phase2_test.csv'
_maDataset = maDataset(csvDataFile)
_maDataset.__len__()

2763

- Fem com si el dataset de entrenament serà el de test, només per veure com va. 
    - Perquè la màquina es petita.

In [9]:
# instantiate the dataloader
batch_size, num_workers = 4, 8
_dataLoader = DataLoader(_maDataset, batch_size = batch_size, shuffle = True, num_workers = num_workers)

- shuffle true, ens interesa que a cada epoch tenir un shuffle diferent. tenir batch diferents

### 2. Backbone
- espina dorsal, es fa servir per referisa a la arquitectura del model
- el nostre backbone es una efficientnet
- dins el concepte backbone es separa el classificador final. 
- backbone es fins el convolutional head i després s'afegeix el classificador
- la fc -> fully conected layer, no està inclosa en el backbone. 
- cada persona se'l ajusta com vol

In [10]:
_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [11]:
import timm

- abans toarch load i carregavem model .pth, model.
- aquí partim de la arquitectura pre entreada de la gent (com la van entrenar els seus autors).
- Això ho fem amb el timm &rarr; hi han moltes arquitectures guardades.

In [12]:
# load the model
model = timm.create_model(
        'tf_efficientnetv2_s',
    
        # Si volem partir del pre entrenament dels autors o no
        pretrained = True,
        num_classes = len(_classes),
        global_pool = 'avg'
    )

- pensar-ho com si la extracció d'atributs que ha fet aquesta gent ens serveix o no
- les fotos que han fet srvir són molt igual totes (taules, cadires, gats, etc)
- tot el procesament d'atributs que la xarxa ha aprés, podem pensar que es extrapolable a objectes del nostre món. 
- si en comptes de analitzar fotos de imatges del nostre mon, analitzem fotos de cel·lules per detectar càncer
- pot no tenir res a veure amb objectes del mon
- es posible que si partim de una pre enetrenada costi fer un fine tinning
- un escaner del cervell
- asumim pels mosquits va bé
- hi ha un altre factor, si hi ha una xarxa de 0, pesos aleatoris, perque volem una xarxa molt específica de mosquits, amb 2000 fotos ni 10000 en fem prou, necesitem una altre ordre de magnitud, fer fer convergir la xarxa cap a un bon resultat. Si les tenim, la xarxa serà molt més eificents. 
- num_classes: el efficentnet té moltes clases.
    - això el que farà en termes de arquitectura, canvia les neurones de sortida, en tenia 10000 perque hi havia 10000 classes, nosaltres li indiquem les que volem. En volem 6. No 7 perquè en el nostre set d'entrenament no li passarem fotos sense mosquit.
- Els pesos que es toquen en el fine tuning només es fa en aquesta ultima capa i pesos, els altres ja ens els creiem.
- els altre pesos no els toquem.
- Lu correcte sería posar imatges amb background i etiqueta ?? i posar-ho al dataset i afegir aquesta neurona 7 per les que no sap identificar.
- global_pool: això ve a dir com farem l'agregat dels arros de la funció d'erro en el batch.
    - podem fer suma, promig, el maxim
    - com s'agreguene els errors en la funció de batch, aquí fem un promig. 
    - vas provant 
    - provem avg, i després provem max o sum
    - cada canvi són 3 dies d'entrenament, cada canvi es a nivell de temps i energia.
- quan parlavem de pooling, les capes qeu fan el kernel de dimensió 1, és el mateix concepte però amb un altre significat.

In [13]:
# Generem el model amb aquesta arquitectura i el classificador i el passem a la gpu:
model = model.to(_device)

### 3. Backpropagation

#### loss function

In [14]:
loss_function = torch.nn.CrossEntropyLoss()

In [15]:
loss_function

CrossEntropyLoss()

- Quan fem train, hem de contrastar l'output per actualitzar els pesos. 
- Funció de loss: 
    - torch nn : hi ha varies funcions de loss. 
    - error mean: $|y_i+y\hat{_i}|$, mean squared error, el mateix al quadrat en comptes de error abs. 
    - això va bé per models de regressió. 
    - en cas de classificació aquestes mesures no acaben d'anar bé. 
    - lu habitual és: 
        - binnari cross entropy (BCE) -> només 2 categoríes
        - cross entropy (CE) -> baries categories +2
        - hi han més.
        - si tenim una distribució de probabilitat, tenim que la nostre xarxa ens diu: 
        - p = [p1, p2, ...px] -> cada p es una probabilitat, la suma de totes es 1
        - la entropia de una distribució de probabilitat es una mesura de cuant incertasa hi ha en aquesta distribució. 
        - es defineix com: en comptes de la suma
        - sumatori de $p_i$ · log($p_i$)
        - si no hi ha cap incertesa, sabem el que passarà, es perque n'hi ha una classe que es 1. 
        - 1·log(1) + 0·log(0) ... totes 0
        - = 0 -> la incertesa és 0
        - si es una distribució unforme, tots iguals, tots poden soritr amb la mateixa probabilitat = maxima incertesa
        - p = [1/k, 1/k, 1/k ...]
        - 1/k·log(1/k) + 1/k·log(1/k) ... tants cops com k = k · 1/K · log(1/k) = 1 · log(1/k) = és la entropia
        - H(P) = CE/BCE = sumatori de tata el que hi ha a dalt.
        - es una mesura que va bé

#### optimizer
- agafa l'error, calculat per la funció de cost. en el context de nn és fa servir la paraula loss
- si l'error és més gran, s'ajustaran més
- en cada epoch s'ha d'anar ajustant

In [16]:
from torch import optim

In [17]:
# optimizer
lRate = 0.0001
optimizer = optim.Adam(model.parameters(), lr = lRate)

In [18]:
optimizer
# Optimizer té els parametres del model

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)

- modul optim hi han diferents tipus d'optimitzadors. Estocastic gradient descent, adam, etc
- diferents conceptes matemàtics. 
- adaptive optimizer algo (adam) va molt bé. 
    - perque tots els parametres de la xarxa es vagin optimitzant a la vegada. 
    - abans que es desenvoupes l'adam, hi havia casos que una part de la nn s'actualitzava bé i una altre part no. aquest ho balançeja bé. 
- ---
- a adam li passem, tots els parametres del model, quins parametres ha de optimitzar (funció `.parameters()`)
    - no torna un numero de parametres, sino una estructura, quins són i com estan relalcionats
- li passem el learning rate. 
    - Té a veure amb quina mesura actualitzem els pesos en funció de la magnitud dels errors. 
    - learning rate 0.5, si tenim error de x, actualitzem l'error com si hagues sigut de la meitat. perque l'error ve d'uns exemples en concret. 
    - tant percent de credibilitat que li donem als errors que hem generat. Ens creiem una deu milèsima de l'error.
- funciona millor primer learning rate petit i després gran. perque al principi no saps, si li fas creure a la xarxa que no sap res, si està classificant cara o creu, si fas un gran al principi, fas actualitzacions grans molt ràpides, si allò ha converit i canvies ja no s'aren res
- al principi fas petit i després gran. pero la majoria agafa scheduler descendets però per en Joan es un error

#### scheduler
- canvia el learning rate en base una funció

In [19]:
gamma = 1.005
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = gamma)

In [20]:
scheduler

- li passem el optimitzador i el step, a cada pas el modificarà. 
- el parametre important és gamma.
- el que volem es que el learning rate vagi augmentant poc a poc. es la manera de seguir apretant la xarxa.
- Podem fer un plot del learning rate.

### 4. Training

In [21]:
import time

In [24]:
%%time
# train
try:
    
    # Posar el model en mode train
    model.train()
    
    # Al de abans no calia fer el with ... posaríem el False i ja està. 
    torch.set_grad_enabled(True)
    
    # mostrem totes les imatges dues vegades prque sino tarda molt
    # hi hauran casos que la xarxa apendra más ràpid, més lent, etc, ensayo i error.
    max_epochs = 4
    
    # loop per totes les epoch
    for epoch in range(max_epochs):
        # Variables que anirem veient el loss que tenim en cada epoch per anar fent control (no funciona)
        train_loss, train_match = .0, .0
        
        # Mirar el que tarda la epoc
        start_time = time.time()
        
        # Iterem per el data loader, aquest ens anirà donant batch de imatges fins que ens les donarà totes.
        # Iterar per el dataset a base de batch
        for batch in _dataLoader:
            
            # per cada batch
            
            # +++ forward pass
            optimizer.zero_grad() # S'han de posar a 0
            inputs = batch['image'].to(_device) # del lot agafem les imatges, passem a la gpu
            output = model(inputs) # Fem inferència (però amb uns pesos aleatoris)
            
            # +++ loss
            labels = torch.cat(tuple(batch['label']), dim = 0).to(_device) # Agafem les etiquetes, que hem posat en el dataset, quan hem definit la classe, s'ha de fer amb tensors, però no està amb la dimensió que toca
            batch_loss = loss_function(output, labels) # les etiquetes són numeros no strings i representen les neurones d'activació (si es label 3, la més activada ha de ser la 3)
            
            
            # +++ backpropagation
            batch_loss.backward() # La funció de loss es un objecte que té un metode
            optimizer.step() # step fa el update, el step ja té el learning rate
            
            # El optimizer té els parametres del model i és així com sap on s'han d'anar a actualitzar
            
            # +++ evaluation
            
            # Ho convertim a predicccions i fem el hard prediction
            # _ es per algo que no volem, ho posa per veure que vagi baixant i així pot parar l'entrenament si no va bé
            _, preds = torch.max(torch.nn.functional.softmax(output, dim = 1), dim = 1)
            
            # com que fem un average del loss, el multipliquem per el shape per tenir un loss més real
            # Interesa com va evolucionant, ens importa la tendència.
            # inputs és el tamany del batch, si posem la mida del batch no perque es un diccionari.
            train_loss += batch_loss.data * inputs.shape[0]
            
            # Accuracy, summar quants cops les nostres prediccions són correctes
            train_match += torch.sum(preds.data == labels.data)

        print('+++ epoch {:3d}, {:6.4f}s Train- Loss: {:.4f} Acc: {:.4f}\n'.format(epoch, (time.time() -start_time), train_loss.item() /_maDataset.__len__(), train_match.item() /_maDataset.__len__()), end = '')
        
except BaseException as err:    
    print(f"+++ batch_inference() {type(err).__name__}, {err}")


+++ epoch   0, 260.9493s Train- Loss: 0.2328 Acc: 0.9160
+++ epoch   1, 262.0361s Train- Loss: 0.3662 Acc: 1.8683
+++ epoch   2, 262.1921s Train- Loss: 0.4917 Acc: 2.8274
+++ batch_inference() KeyboardInterrupt, 
CPU times: user 15min 47s, sys: 5.09 s, total: 15min 52s
Wall time: 15min 54s


- si mirem tenim 4/8 podríem augmentar el batch size
- però 8 min per fer 2 epoch
- ---
- aquest script amb tensorflow són dues línies de codi.

### 5. Evaluate

In [23]:
df_pred = pd.DataFrame(preds)
df_pred.head()

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
df_ = pd.merge(_maDataset.df, df_pred, how = 'inner', on = 'img_fName')
df_.head()

In [ ]:
df_.groupby('class_label').pred_label.value_counts()

#### Classification

In [ ]:
_, axs = plt.subplots(1, 3, figsize = (15, 7), sharey = True)
for i, norm in enumerate([None, 'true', 'pred']):
    ConfusionMatrixDisplay.from_predictions(
        df_.class_label,
        df_.pred_label,
        normalize = norm,
        ax = axs[i],
        display_labels = ['??', 'aeg', 'alb', 'ano', 'clx', 'cul', 'j/k'],
        cmap = 'GnBu',
        colorbar = None
    )
plt.tight_layout()

In [ ]:
avrgs = ['macro', 'micro', 'weighted']
pd.DataFrame([prf(df_.class_label, df_.pred_label, average = mode, zero_division = 0)[:3] for mode in avrgs], columns = ['precision', 'recall', 'f-score'], index = avrgs)

#### check predictions

In [ ]:
i = -1

In [ ]:
i += 1
row = df_.iloc[i]
pilImg = Image.open('%s/%s' %(_imgRoot, row.img_fName))
imgdrw = ImageDraw.Draw(pilImg)
imgdrw.rectangle([(row.bbx_xtl, row.bbx_ytl), (row.bbx_xbr, row.bbx_ybr)], outline = 'blue', width = 2)
plt.imshow(pilImg)
plt.axis('off');
print('+++%3d %s - %s / %s' %(i, row.img_fName, row.class_label, row.pred_label))

In [ ]:
chk = df_[df_.pred_label == '??']
len(chk)

In [ ]:
j = -1

In [ ]:
j += 1
row = chk.iloc[j]
pilImg = Image.open('%s/%s' %(_imgRoot, row.img_fName))
imgdrw = ImageDraw.Draw(pilImg)
imgdrw.rectangle([(row.bbx_xtl, row.bbx_ytl), (row.bbx_xbr, row.bbx_ybr)], outline = 'blue', width = 8)
plt.imshow(pilImg)
plt.axis('off');
print('+++%3d - %s / %s' %(i, row.class_label, row.pred_label))